In [1]:
# import libraries
import pandas as pd
import sqlite3 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import itertools
import warnings; warnings.simplefilter('ignore')
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV  
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from pandas import set_option
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold 
from sklearn.model_selection import RandomizedSearchCV  
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
pd.set_option('display.max_columns', 1000)
%matplotlib inline 
plt.style.use('ggplot') 

### Import Clean Data CSV - Create Dummies

In [4]:
df = pd.read_csv('clean_data.csv', index_col=None)

In [5]:
df['no_show']=np.where(df['no_show']=='No',0,1)

In [7]:
df.drop(['patient_id', 'appointment_id'], axis=1, inplace=True)

In [9]:
df = pd.get_dummies(df, columns=['gender', 'neighbourhood'], drop_first=True, prefix_sep='_')

In [10]:
df['scheduled_day'] = pd.to_datetime(df['scheduled_day'])
df['appointment_day'] = pd.to_datetime(df['appointment_day'])

In [12]:
df.head()

,scheduled_day,appointment_day,age,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show,hdi,gender_M,neighbourhood_ANDORINHAS,neighbourhood_ANTÔNIO HONÓRIO,neighbourhood_ARIOVALDO FAVALESSA,neighbourhood_BARRO VERMELHO,neighbourhood_BELA VISTA,neighbourhood_BENTO FERREIRA,neighbourhood_BOA VISTA,neighbourhood_BONFIM,neighbourhood_CARATOÍRA,neighbourhood_CENTRO,neighbourhood_COMDUSA,neighbourhood_CONQUISTA,neighbourhood_CONSOLAÇÃO,neighbourhood_CRUZAMENTO,neighbourhood_DA PENHA,neighbourhood_DE LOURDES,neighbourhood_DO CABRAL,neighbourhood_DO MOSCOSO,neighbourhood_DO QUADRO,neighbourhood_ENSEADA DO SUÁ,neighbourhood_ESTRELINHA,neighbourhood_FONTE GRANDE,neighbourhood_FORTE SÃO JOÃO,neighbourhood_FRADINHOS,neighbourhood_GOIABEIRAS,neighbourhood_GRANDE VITÓRIA,neighbourhood_GURIGICA,neighbourhood_HORTO,neighbourhood_ILHA DAS CAIEIRAS,neighbourhood_ILHA DE SANTA MARIA,neighbourhood_ILHA DO BOI,neighbourhood_ILHA DO FRADE,neighbourhood_ILHA DO PRÍNCIPE,neighbourhood_ILHAS OCEÂNICAS DE TRINDADE,neighbourhood_INHANGUETÁ,neighbourhood_ITARARÉ,neighbourhood_JABOUR,neighbourhood_JARDIM CAMBURI,neighbourhood_JARDIM DA PENHA,neighbourhood_JESUS DE NAZARETH,neighbourhood_JOANA D´ARC,neighbourhood_JUCUTUQUARA,neighbourhood_MARIA ORTIZ,neighbourhood_MARUÍPE,neighbourhood_MATA DA PRAIA,neighbourhood_MONTE BELO,neighbourhood_MORADA DE CAMBURI,neighbourhood_MÁRIO CYPRESTE,neighbourhood_NAZARETH,neighbourhood_NOVA PALESTINA,neighbourhood_PARQUE INDUSTRIAL,neighbourhood_PARQUE MOSCOSO,neighbourhood_PIEDADE,neighbourhood_PONTAL DE CAMBURI,neighbourhood_PRAIA DO CANTO,neighbourhood_PRAIA DO SUÁ,neighbourhood_REDENÇÃO,neighbourhood_REPÚBLICA,neighbourhood_RESISTÊNCIA,neighbourhood_ROMÃO,neighbourhood_SANTA CECÍLIA,neighbourhood_SANTA CLARA,neighbourhood_SANTA HELENA,neighbourhood_SANTA LUÍZA,neighbourhood_SANTA LÚCIA,neighbourhood_SANTA MARTHA,neighbourhood_SANTA TEREZA,neighbourhood_SANTO ANDRÉ,neighbourhood_SANTO ANTÔNIO,neighbourhood_SANTOS DUMONT,neighbourhood_SANTOS REIS,neighbourhood_SEGURANÇA DO LAR,neighbourhood_SOLON BORGES,neighbourhood_SÃO BENEDITO,neighbourhood_SÃO CRISTÓVÃO,neighbourhood_SÃO JOSÉ,neighbourhood_SÃO PEDRO,neighbourhood_TABUAZEIRO,neighbourhood_UNIVERSITÁRIO,neighbourhood_VILA RUBIM
0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,0,1,0,0,0,0,0,0.939,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,0,0,0,0,0,0,0,0.939,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,0,0,0,0,0,0,0,0.961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,0,0,0,0,0,0,0,0.667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,0,1,1,0,0,0,0,0.939,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Train/ Test Split

In [13]:
# Create matrix of features
Xc = df.drop(['no_show', 'scheduled_day', 'appointment_day'], axis = 1) # grabs everything else but 'no_show'

# Create target variable
yc = df['no_show'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_colsC = Xc.columns

In [14]:
# Use x and y variables to split the training data into train and test set then scale that data

X_trainC, X_testC, y_trainC, y_testC = train_test_split(Xc, yc, random_state=1)

scalerC = StandardScaler()  
scalerC.fit(X_trainC)

X_trainC = scalerC.transform(X_trainC)  
X_testC = scalerC.transform(X_testC)

### Baseline models on clean data

#### KNN

In [215]:
knn = KNeighborsClassifier(n_neighbors=9)

In [216]:
knn.fit(X_trainC, y_trainC)

knn_preds = knn.predict(X_testC)

knn_f1 = metrics.f1_score(y_testC, knn_preds)

knn_acc = metrics.accuracy_score(y_testC, knn_preds)

print('Test F1 score: ', knn_f1)
print('Test Accuracy score: ', knn_acc)

Test F1 score:  0.11617278490343506
Test Accuracy score:  0.7830414012738853


#### Logisitic Regression 

In [217]:
lr = LogisticRegression(class_weight='balanced')

In [218]:
lr.fit(X_trainC, y_trainC)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [219]:
lr_preds = lr.predict(X_testC)

lr_f1 = metrics.f1_score(y_testC, lr_preds)

lr_acc = metrics.accuracy_score(y_testC, lr_preds)

print('Test F1 score: ', lr_f1)
print('Test Accuracy score: ', lr_acc)

Test F1 score:  0.35339052767657775
Test Accuracy score:  0.6017660683265779


#### Decision Tree

In [231]:
dtcC = DecisionTreeClassifier(max_depth=5, class_weight='balanced')

dtcC.fit(X_trainC, y_trainC)

dtcC_preds  = dtcC.predict(X_testC)

dtcC_f1 = metrics.f1_score(y_testC, dtcC_preds)
dtcC_acc = metrics.accuracy_score(y_testC, dtcC_preds)

print('Test F1 score: ', dtcC_f1)
print('Test Accuracy score: ', dtcC_acc)

Test F1 score:  0.3550227104679413
Test Accuracy score:  0.5580486392588303


#### Random Forest

In [15]:
rfcC = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=9, max_features=3, class_weight= 'balanced')

In [16]:
rfcC.fit(X_trainC, y_trainC)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=9, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [233]:
rfcC_preds = rfcC.predict(X_testC)

rfcC_f1 = metrics.f1_score(y_testC, rfcC_preds)
rfcC_acc = metrics.accuracy_score(y_testC, rfcC_preds)

print('Test F1 score: ', rfcC_f1)
print('Test Accuracy Score: ', rfcC_acc)

Test F1 score:  0.35033401849948614
Test Accuracy Score:  0.6339751013317892


### Models with Engineered Features

#### Import Features CSV - Create Dummies

In [162]:
feat_df = pd.read_csv('feature_data_clean.csv')

In [163]:
feat_df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show,hdi,day_of_week,waiting_days,waiting_categories,current_year,year_birth,generation,at_risk
0,2.987250e+13,5642903,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,1,0.939,Friday,0,same day: 0,2017,1955,boomer,1
1,5.589978e+14,5642503,M,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,1,0.939,Friday,0,same day: 0,2017,1961,boomer,0
2,4.262962e+12,5642549,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,1,0.961,Friday,0,same day: 0,2017,1955,boomer,0
3,8.679512e+11,5642828,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,1,0.667,Friday,0,same day: 0,2017,2009,zoomer,0
4,8.841186e+12,5642494,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,1,0.939,Friday,0,same day: 0,2017,1961,boomer,1


In [164]:
feat_df.drop(['patient_id', 'appointment_id', 'scheduled_day', 'appointment_day', 'current_year', 'year_birth', 
              'sms_received', 'hipertension', 'neighbourhood'], axis=1, inplace=True)

In [165]:
feat_df = pd.get_dummies(feat_df, columns=['gender', 'day_of_week', 'waiting_categories', 'generation', 'at_risk'], drop_first=True, prefix_sep='_')

In [166]:
feat_df.head()

,age,scholarship,diabetes,alcoholism,handcap,no_show,hdi,waiting_days,gender_M,day_of_week_Monday,day_of_week_Saturday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,waiting_categories_month: 16-30,waiting_categories_quarter: 31-90,waiting_categories_same day: 0,waiting_categories_season: 91-180,waiting_categories_short: 1-3,waiting_categories_week: 4-7,generation_boomer,generation_greatest,generation_millennial,generation_silent,generation_x,generation_zoomer,at_risk_1
0,62,0,0,0,0,1,0.939,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1
1,56,0,0,0,0,1,0.939,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,62,0,0,0,0,1,0.961,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,8,0,0,0,0,1,0.667,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,56,0,1,0,0,1,0.939,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


#### Train/Test Split

In [167]:
# Create matrix of features
Xf = feat_df.drop(['no_show'], axis = 1) # grabs everything else but 'no_show'

# Create target variable
yf = feat_df['no_show'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_colsF = Xf.columns

In [168]:
# Use x and y variables to split the training data into train and test set then scale that data

X_trainF, X_testF, y_trainF, y_testF = train_test_split(Xf, yf, random_state=1)

scalerF = StandardScaler()  
scalerF.fit(X_trainF)

X_trainF = scalerF.transform(X_trainF)  
X_testF = scalerF.transform(X_testF)

#### KNN with Features

In [224]:
knnF = KNeighborsClassifier(n_neighbors=9)

In [226]:
knnF.fit(X_trainF, y_trainF)

knnF_preds = knnF.predict(X_testF)

knnF_f1 = metrics.f1_score(y_testF, knnF_preds)

knnF_acc = metrics.accuracy_score(y_testF, knnF_preds)

print('Test F1 score: ', knnF_f1)
print('Test Accuracy score: ', knnF_acc)

Test F1 score:  0.8723301480625852
Test Accuracy score:  0.779993485577793


#### Logisitic Regression with Features 

In [227]:
lrF = LogisticRegression(class_weight='balanced')

In [228]:
lrF.fit(X_trainF, y_trainF)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [229]:
lrF_preds = lrF.predict(X_testF)

lrF_f1 = metrics.f1_score(y_testF, lrF_preds)

lrF_acc = metrics.accuracy_score(y_testF, lrF_preds)

print('Test F1 score: ', lr_f1)
print('Test Accuracy score: ', lrF_acc)

Test F1 score:  0.35339052767657775
Test Accuracy score:  0.5728710506315371


#### Decision Tree with Features

In [232]:
dtcF = DecisionTreeClassifier(max_depth=9, class_weight='balanced')

dtcF.fit(X_trainF, y_trainF)

dtcF_preds  = dtcF.predict(X_testF)

dtcF_f1 = metrics.f1_score(y_testF, dtcF_preds)
dtcF_acc = metrics.accuracy_score(y_testF, dtcF_preds)

print('Test F1 score: ', dtcF_f1)
print('Test Accuracy score: ', dtcF_acc)

Test F1 score:  0.6672419248137601
Test Accuracy score:  0.581303608266078


#### Random Forest with Features

In [169]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfcF = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=9, max_features=3, class_weight='balanced')

In [170]:
#fit the model to the training data
rfcF.fit(X_trainF, y_trainF)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=9, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [234]:
rfcF_preds = rfcF.predict(X_testF)

rfcF_f1 = metrics.f1_score(y_testF, rfcF_preds)
rfcF_acc = metrics.accuracy_score(y_testF, rfcF_preds)

print('Test F1 score: ', rfcF_f1)
print('Test Accuracy Score: ', rfcF_acc)

Test F1 score:  0.6486709252009066
Test Accuracy Score:  0.568057616445297


In [172]:
confusion_matrix(y_testF, rfcF_preds, labels = [0,1])

array([[ 4678,   862],
       [11073, 11018]])

### Random Forest Grid Search

In [174]:
#create a dictionary of all the parameters you want to tune
param_gridF = { 
    'n_estimators': [450],
    'max_features': ['auto'],
    'max_depth' : [50],
    'criterion' :['gini'],
    'class_weight' : ['balanced']
}

In [175]:
#create a grid search object and fit it to the data

CV_rfcF = GridSearchCV(estimator=rfcF, param_grid=param_gridF, cv= 5, verbose=3, n_jobs= -1)
CV_rfcF.fit(X_trainF, y_trainF)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=9,
                                              max_features=3,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=1,
                                     

In [176]:
### Identify the best params 
print(CV_rfcF.best_params_)
print(CV_rfcF.best_estimator_)


#Identify the best score during fitting with cross-validation
print(CV_rfcF.best_score_)


{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 50, 'max_features': 'auto', 'n_estimators': 450}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=450,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
0.7718570735270565


In [177]:
#predict on the test set
test_preds_CV_rfcF = CV_rfcF.best_estimator_.predict(X_testF)

# checking accuracy and F1
print("Accuracy:",metrics.accuracy_score(y_testF, test_preds_CV_rfcF))
print("F1:",metrics.f1_score(y_testF, test_preds_CV_rfcF))


Accuracy: 0.770619955846694
F1: 0.862979937737807


In [111]:
confusion_matrix(y_testF, test_preds_CV_rfcF, labels = [0,1])

array([[ 2600,  2940],
       [ 4437, 17654]])

### XGBoost

In [178]:
import xgboost as xgb

In [179]:
xgb.XGBClassifier()

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=False, verbosity=None)

In [180]:
xg_clf = xgb.XGBClassifier(objective ='binary:logistic', 
                           colsample_bytree = 0.3, 
                           subsample = 0.5,
                           learning_rate = 0.1,
                           max_depth = 5, 
                           alpha = 1, 
                           n_estimators = 1000,
                           silent=1)

In [181]:
xg_clf.fit(X_trainF,y_trainF)


XGBClassifier(alpha=1, base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=1,
              reg_lambda=1, scale_pos_weight=1, silent=1, subsample=0.5,
              tree_method=None, validate_parameters=False, verbosity=None)

In [182]:
xg_preds = xg_clf.predict(X_testF)


xg_test_f1 = f1_score(y_testF, xg_preds)
xg_test_acc = accuracy_score(y_testF, xg_preds)

print("Accuracy: %f" % (xg_test_acc))
print("F1: %f" % (xg_test_f1))

Accuracy: 0.798270
F1: 0.886560


### Import Feature CSV - Preparing for XGBoost Grid Search

In [185]:
cv_df = pd.read_csv('feature_data_clean.csv')

In [186]:
cv_df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show,hdi,day_of_week,waiting_days,waiting_categories,current_year,year_birth,generation,at_risk
0,2.987250e+13,5642903,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,1,0.939,Friday,0,same day: 0,2017,1955,boomer,1
1,5.589978e+14,5642503,M,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,1,0.939,Friday,0,same day: 0,2017,1961,boomer,0
2,4.262962e+12,5642549,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,1,0.961,Friday,0,same day: 0,2017,1955,boomer,0
3,8.679512e+11,5642828,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,1,0.667,Friday,0,same day: 0,2017,2009,zoomer,0
4,8.841186e+12,5642494,F,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,1,0.939,Friday,0,same day: 0,2017,1961,boomer,1


In [187]:
cv_df.drop(['patient_id', 'appointment_id', 'scheduled_day', 'appointment_day', 'current_year', 'year_birth', 
              'sms_received', 'hipertension', 'neighbourhood'], axis=1, inplace=True)

In [188]:
cv_df = pd.get_dummies(cv_df, columns=['gender', 'day_of_week', 'waiting_categories', 'generation'], drop_first=True, prefix_sep='_')

In [189]:
cv_df.head()

,age,scholarship,diabetes,alcoholism,handcap,no_show,hdi,waiting_days,at_risk,gender_M,day_of_week_Monday,day_of_week_Saturday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,waiting_categories_month: 16-30,waiting_categories_quarter: 31-90,waiting_categories_same day: 0,waiting_categories_season: 91-180,waiting_categories_short: 1-3,waiting_categories_week: 4-7,generation_boomer,generation_greatest,generation_millennial,generation_silent,generation_x,generation_zoomer
0,62,0,0,0,0,1,0.939,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,56,0,0,0,0,1,0.939,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,62,0,0,0,0,1,0.961,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,8,0,0,0,0,1,0.667,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,56,0,1,0,0,1,0.939,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


#### Train/Test Split

In [190]:
# Create matrix of features
Xcv = cv_df.drop(['no_show'], axis = 1) # grabs everything else but 'no_show'

# Create target variable
ycv = cv_df['no_show'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_colscv = Xcv.columns

In [191]:
# Use x and y variables to split the training data into train and test set then scale that data

X_trainGBCV, X_testGBCV, y_trainGBCV, y_testGBCV = train_test_split(Xcv, ycv, random_state=1)


In [192]:
X_trainGBCV.head()

,age,scholarship,diabetes,alcoholism,handcap,hdi,waiting_days,at_risk,gender_M,day_of_week_Monday,day_of_week_Saturday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,waiting_categories_month: 16-30,waiting_categories_quarter: 31-90,waiting_categories_same day: 0,waiting_categories_season: 91-180,waiting_categories_short: 1-3,waiting_categories_week: 4-7,generation_boomer,generation_greatest,generation_millennial,generation_silent,generation_x,generation_zoomer
103791,44,0,0,0,0,0.648,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
82500,22,0,0,0,0,0.730,40,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
36310,28,0,0,0,0,0.684,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
44969,66,0,0,0,0,0.642,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
7892,16,0,0,0,0,0.911,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


#### Define Model Fit Function

In [193]:
def modelfit(alg, dtrain, predictors, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=5):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % roc_auc_score(dtrain[target], dtrain_predprob))

    return alg
#     feat_imp = pd.Series(alg.get_booster().get_fscore())
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

In [196]:
train  = pd.concat([X_trainGBCV, y_trainGBCV], axis=1)
target = 'no_show'

In [197]:
train.head()

,age,scholarship,diabetes,alcoholism,handcap,hdi,waiting_days,at_risk,gender_M,day_of_week_Monday,day_of_week_Saturday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,waiting_categories_month: 16-30,waiting_categories_quarter: 31-90,waiting_categories_same day: 0,waiting_categories_season: 91-180,waiting_categories_short: 1-3,waiting_categories_week: 4-7,generation_boomer,generation_greatest,generation_millennial,generation_silent,generation_x,generation_zoomer,no_show
103791,44,0,0,0,0,0.648,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
82500,22,0,0,0,0,0.730,40,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
36310,28,0,0,0,0,0.684,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
44969,66,0,0,0,0,0.642,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
7892,16,0,0,0,0,0.911,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


#### Define Predictors

In [201]:
predictors = [x for x in train.columns if x not in [target]]
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.6,
 colsample_bytree=0.3,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

alg = modelfit(xgb1, train, predictors, target)


Model Report
Accuracy : 0.7977
AUC Score (Train): 0.735910


In [203]:
GBCV_preds = alg.predict(X_testGBCV)


GBCV_test_f1 = f1_score(y_testGBCV, GBCV_preds)
GBCV_test_acc = accuracy_score(y_testGBCV, GBCV_preds)

print("Accuracy: %f" % (GBCV_test_acc))
print("F1: %f" % (GBCV_test_f1))

Accuracy: 0.799537
F1: 0.888563


### XGBoost GridSearch

In [207]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': [1000],
              'learning_rate': [0.1,0.05,0.01],
              'max_depth': [3, 4, 5],
              'colsample_bytree': [0.3,0.4],
              'min_child_weight': [1, 2]
             }

In [208]:
# initiate the Gridsearch model
gsearch1 = GridSearchCV(
    estimator = clf_xgb,
    param_grid = param_dist, 
    scoring='f1',
    n_jobs=-1,
    verbose=1,
    iid=False, 
    cv=5)


In [209]:
gsearch1.fit(train[predictors],train[target])


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 52.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estim...
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
             iid=

In [210]:
gsearch1.cv_results_

{'mean_fit_time': array([54.14129305, 53.24256678, 64.1790802 , 64.44030876, 77.90618272,
        76.78548827, 53.9077333 , 53.80316124, 64.67739158, 64.97236657,
        76.21289964, 75.96980467, 54.69985371, 54.60298219, 65.80742803,
        67.00067024, 78.42278271, 77.86528535, 62.55372658, 62.4565475 ,
        76.66182752, 76.36993127, 93.29353271, 90.89524355, 59.04274464,
        55.34860864, 67.8985919 , 67.79450803, 80.68536034, 81.98421249,
        56.49174476, 56.18291059, 69.19210992, 69.49902096, 84.9568718 ,
        80.97280955]),
 'std_fit_time': array([0.3027864 , 0.08468444, 0.1754887 , 0.45927223, 0.90754506,
        1.29561176, 0.61923267, 0.32058734, 0.31149791, 0.26660125,
        0.85896345, 0.2597097 , 0.16621278, 0.24956764, 0.35208764,
        1.23963486, 0.76951528, 0.26756489, 0.21399704, 0.17187626,
        0.3184001 , 0.23038215, 1.0471693 , 0.2659518 , 2.52352565,
        0.12166904, 0.20501966, 0.10460533, 0.14578107, 0.77318961,
        0.26467395, 0.226

In [211]:
gsearch1.best_params_

{'colsample_bytree': 0.3,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 2,
 'n_estimators': 1000}

In [212]:
gsearch1.best_score_

0.8874240594530212

In [214]:
gs_preds = gsearch1.best_estimator_.predict(X_testGBCV)


gs_test_f1 = f1_score(y_testGBCV, gs_preds)
gs_test_acc = accuracy_score(y_testGBCV, gs_preds)

print("Accuracy: %f" % (gs_test_acc))
print("F1: %f" % (gs_test_f1))

Accuracy: 0.799609
F1: 0.888625
